In [14]:
pip install -U sentence-transformers

You should consider upgrading via the '/Users/nasim/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install openpyxl

You should consider upgrading via the '/Users/nasim/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, util, SentenceTransformer, LoggingHandler
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers.evaluation import InformationRetrievalEvaluator, BinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
from datetime import datetime
from zipfile import ZipFile
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import logging
import csv
import sys
import torch
import math
import gzip
import os
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print deLoggingHandlerbug information to stdout

#You can specify any huggingface/transformers pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-base
model_name = 'allenai/scibert_scivocab_uncased'
batch_size = 2
num_epochs = 10
max_seq_length = 256
use_cuda = torch.cuda.is_available()

In [18]:
bi_encoder_path = 'bi-encoder/model'

In [19]:
def get_similarity_vectors(vector1, vector2):
    cos_sim = util.pytorch_cos_sim(vector1, vector2).item()
    return cos_sim

In [20]:
annotations_df = pd.read_excel('Annotations.xlsx', engine='openpyxl',)

In [21]:
annotations_df.tail()

,SN,Source,Target,Classification
276,277,We propose an over-sampling approach in which ...,"""Poisson likelihood models have been prevalent...",0
277,278,We propose an over-sampling approach in which ...,"In this section, we show a connection between ...",0
278,279,We propose an over-sampling approach in which ...,"""In this chapter, we describe topic models, pr...",0
279,280,We propose an over-sampling approach in which ...,While the increasing impact of advanced black-...,0
280,281,We propose an over-sampling approach in which ...,"""First, we propose an attention mechanism that...",0


In [22]:
sentence1 = annotations_df.Source[2]
sentence2 = annotations_df.Target[2]

In [23]:
bi_encoder = SentenceTransformer(bi_encoder_path)
embeddings1 = bi_encoder.encode(sentence1, convert_to_tensor=True)
embeddings2 = bi_encoder.encode(sentence2, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
print(cosine_scores)

2021-06-22 18:47:23 - Load pretrained SentenceTransformer: bi-encoder/model
2021-06-22 18:47:23 - Load SentenceTransformer from folder: bi-encoder/model
2021-06-22 18:47:25 - Use pytorch device: cpu


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
ir_queries = {} 
ir_corpus = {}             
ir_relevant_docs = {}       


for index, row in annotations_df.iterrows():
    ir_queries[row['SN']] = row['Source']
    ir_corpus[row['SN']] = row['Target']
    if row['Classification'] == 1.0:
        ir_relevant_docs[row['SN']] = set([row['SN']])


In [ ]:
#Binary classification evaluator
test_sentences1 = []
test_sentences2 = []
test_labels = []

for index, row in annotations_df.iterrows():
    test_sentences1.append(row['Source'])
    test_sentences2.append(row['Target'])
    test_labels.append(int(row['Classification']))
    
evaluator1 = BinaryClassificationEvaluator(test_sentences1, test_sentences2, test_labels)
bi_encoder.evaluate(evaluator1)   

In [ ]:
#IR Evaluation
logging.info("Read ARXIV test dataset")

evaluator2 = InformationRetrievalEvaluator(ir_queries, ir_corpus, ir_relevant_docs)
bi_encoder.evaluate(evaluator2)
    

In [ ]:
paragraph1 = ['In most cases the output of an ANN is a nonlinear function of its parameters because of the choice of its activation functions (e.g., sigmoid or tanh function). As a result, it is no longer straightforward to derive a solution for w that is guaranteed to be globally optimal. Greedy algorithms such as those based on the gradient descent method have been developed to efficiently solve the optimization problem. The weight update formula used by the gradient descent method']
paragraph2 = ['With the increase in available data parallel machine learning has become an increasingly pressing problem. In this paper we present the first parallel stochastic gradient descent algorithm including a detailed analysis and experimental evidence. Unlike prior work on parallel optimization algorithms [5, 7] our variant comes with parallel acceleration guarantees and it poses no overly tight latency constraints, which might only be available in the multicore setting. Our analysis introduces a novel proof technique — contractive mappings to quantify the speed of convergence of parameter distributions to their asymptotic limits. As a side effect this answers the question of how quickly stochastic gradient descent algorithms reach the asymptotically normal regime']

embeddings1 = bi_encoder.encode(paragraph1, convert_to_tensor=True)
embeddings2 = bi_encoder.encode(paragraph2, convert_to_tensor=True)

cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
print(cosine_scores.item())


In [ ]:
paragraphs_df = pd.read_excel('paragraphs.xlsx', engine='openpyxl',)

In [ ]:
paragraphs_df.head()

In [ ]:
source_paras = paragraphs_df['Source paragraph'].tolist()
target_paras = paragraphs_df['Target Paragraph'].tolist()
similarity = paragraphs_df['Similarity'].tolist()

In [ ]:
source = []
target = []
for s, t in zip(source_paras, target_paras):
    source.append(s.replace("\n", " "))
    target.append(t.replace("\n", " "))

In [ ]:
#Evaluate paragraphs on binary classification evaluator
para_eval = BinaryClassificationEvaluator(source, target, similarity)
bi_encoder.evaluate(para_eval)   

In [ ]:
s_embeddings = bi_encoder.encode(source, convert_to_tensor=True)
t_embeddings = bi_encoder.encode(target, convert_to_tensor=True)
c_scores = util.pytorch_cos_sim(s_embeddings, t_embeddings)

In [ ]:
p_scores = []
for i in range(len(source)): 
    p_scores.append(c_scores[i][i].item())

In [ ]:
paragraphs_df['predicted score'] = p_scores

In [ ]:
paragraphs_df

In [ ]:
# matplotlib histogram
plt.hist(paragraphs_df['predicted score'], color = 'blue', edgecolor = 'black',
         bins = 10)
# Add labels
plt.title('Histogram of scores distribution')
plt.xlabel('similarity score')
plt.ylabel('COUNT')